In [ ]:
# объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх
# отсортировать по убыванию столбец с отзывами, затем с оценками

select id,
        reviews_per_month,
        review_scores_rating
        
from listings
where toFloat32OrNull(review_scores_rating) > (select avg(toFloat32OrNull(review_scores_rating)) as rating_avg 
                                                from listings )
    and reviews_per_month < 3
                                   
order by reviews_per_month desc , toFloat64OrNull(review_scores_rating) desc                                    
limit 100


In [ ]:
#и id хоста и идентификаторы объявлений о сдаче отдельных комнат , для которых расстояние до центра оказалось меньше среднего
#52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

select id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat32(longitude) , toFloat32(latitude)) as distance_avg
from listings 
where distance_avg <  #отбираем объекты с расстоянием меньше среднего
                    (
                    select avg(geoDistance(13.4050, 52.5200, toFloat32(longitude) , toFloat32(latitude))) as distance_avg
                        from listings
                        where room_type = 'Private room'
                    ) and
        room_type = 'Private room'
order by distance_avg desc
limit 6

In [ ]:
# объявления с жильем:
# 1) на расстоянии от центра меньше среднего
# 2) обойдутся дешевле 100$ в день (price + cleaning_fee(указана за неделю))
# 3) последние отзывы (last_review), начиная с 1 сентября 2018 года
# 4) имеют WiFi в списке удобств (amenities)

select id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat32(longitude) , toFloat32(latitude)) as distance_avg,
    toFloat32OrNull(review_scores_rating) as review_scores_rating
from listings 
where distance_avg < (select avg(geoDistance(13.4050, 52.5200, toFloat32(longitude) , toFloat32(latitude))) as distance_avg
                       from listings) --имеем число  
        and (toFloat32OrNull(replaceAll(price, '$', '')) + toFloat32OrNull(replaceAll(cleaning_fee, '$', ''))/7) < 100
        and toStartOfMonth(toDateOrNull(last_review)) > '2018-09-01'
        and amenities like '%Wifi%'
order by review_scores_rating desc
limit 6
    

In [ ]:
# найти в таблице calendar_summary те доступные (available='t') объявления,
# у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.

with reviewer_count_avg_table as  #CTE со средним числом пользователей, которые оставили отзыв по одному объекту 
(   select avg(reviewer_count) as reviewer_count_avg
    from(
        select uniqExact(reviewer_id) as reviewer_count
        from reviews 
        group by listing_id
        order by reviewer_count desc
        ) 
)    


select listing_id, uniqExact(reviewer_id) as reviewer_id_count
from (
    select DISTINCT(listing_id)
    from calendar_summary    #условие на выборку уникальных значений из списка доступности объектов
    where available='t'
    GROUP BY listing_id
    ) as l
    
join reviews as r 
    on l.listing_id = r.listing_id
    
group by l.listing_id
having reviewer_id_count > (select reviewer_count_avg from reviewer_count_avg_table) #число оставивших отызв > среднего числа
order by l.listing_id
limit 10


In [ ]:
# разделить всех покупателей на сегменты по стимости среднего чека.

select UserID, 
        avg(Rub) as rub_avg,
case when rub_avg < 5 then 'A'
    when rub_avg >= 5 and rub_avg < 10 then 'B'
    when rub_avg >= 10 and rub_avg < 20 then 'C'
    when rub_avg >= 20 then 'D'
    end as my_type
from checks
group by UserID
order by UserID
limit 10

In [ ]:
#вывести доход, который приносит каждый из сегментов и сколько пользователей содержит в себе

select my_type,
    count(UserID) as users_count,   #вычисляем кол-во покупателей по сегментам
    sum(rub_sum) as rub_total       #вычисляем сумму дохода от покупателей по сегментам
from(  #разделяем покупателей на сегменты по среднему чеку
        select UserID, 
                avg(Rub) as rub_avg,
                r.rub_sum,
        case when rub_avg < 5 then 'A'
            when rub_avg >= 5 and rub_avg < 10 then 'B'
            when rub_avg >= 10 and rub_avg < 20 then 'C'
            when rub_avg >= 20 then 'D'
            end as my_type
        from checks as l
        
        join ( #добавляем столбец с доходом по каждому пользователю
                select UserID, 
                        sum(Rub) as rub_sum
                from checks
                group by UserID
                ) as r
        ON l.UserID = r.UserID
        
        
        group by UserID,rub_sum
        order by UserID
    )
GROUP BY my_type
order by rub_total desc


In [ ]:
#создать колонку с оценкой услови о наличии кухни и "гибкой" системы отмены
#отсортировать новую колонку по возростанию
#выписать host_id из первый строки

select id,
        host_id,
case when multiSearchAnyCaseInsensitive(amenities, ['kitchen']) and cancellation_policy =='flexible' then 'good'
    when multiSearchAnyCaseInsensitive(amenities, ['kitchen']) and cancellation_policy !='flexible' then 'ok'
    else 'not ok'
    end as my_status
from listings 
order by my_status
limit 5

In [ ]:
#создать таблицу

CREATE TABLE test.reviews ENGINE = MergeTree ORDER BY (listing_id, id) (
    listing_id UInt32,
    id  UInt32,
    created_at DateTime('Europe/Moscow'),
    reviewer_id UInt32,
    reviewer_name string,
    comments string
)

In [ ]:
#изменить тип в столбце на дату

ALTER TABLE test.reviews MODIFY COLUMN created_at Date

In [ ]:
#удалить строки с пустыми значениями в столбце
ALTER TABLE test.reviews DELETE WHERE comments=''

In [ ]:
#добавить колонку после колонки "comments"

ALTER TABLE test.reviews ADD COLUMN price Float32 AFTER comments